In [1]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!gdown -O mix.zip --id 1bS57vvFGroKx32aLkuYNTdq0JQNgUWYb

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1bS57vvFGroKx32aLkuYNTdq0JQNgUWYb
To: /content/mix.zip
100% 314M/314M [00:05<00:00, 60.6MB/s]


In [ ]:
!unzip mix.zip 

In [13]:
%cd /content
!mkdir rawdata
%cd rawdata
!mkdir raw_train raw_val
%cd /content

/content
/content/rawdata
/content


In [20]:
import os
import shutil
import random
paths = [i.path for i in os.scandir("/content/mix") if i.path.split(".")[-1] == 'json']
paths = sorted(paths)
random.shuffle(paths)
len(paths)


1383

In [ ]:
trainps, valps = paths[:1200], paths[1200:]
valps

In [26]:
!rm /content/rawdata/raw_val/*
!rm /content/rawdata/raw_train/*

In [27]:
for i in trainps:
    name = i.split("/")[-1]
    iname = i.replace(".json", ".png")
    shutil.copy(i, os.path.join("rawdata/raw_train", name))
    shutil.copy(iname, os.path.join("rawdata/raw_train", iname.split("/")[-1]))


In [28]:
for i in valps:
    name = i.split("/")[-1]
    iname = i.replace(".json", ".png")
    shutil.copy(i, os.path.join("rawdata/raw_val", i.split("/")[-1]))
    shutil.copy(iname, os.path.join("rawdata/raw_val", iname.split("/")[-1]))

In [11]:
!rm -rf /content/rawdata

In [6]:
category_map = {
    "heading": 0,  # person
    "question": 1,  # bicycle
    "quetion": 1,  # bicycle
    "quesition": 1,  # bicycle
    "subquestion": 2,
    "sub_question": 2,
    "subquestioin":2,
    "subquetion":2,
    "choice"  : 3,
    "image"   : 4,
    "table"   : 5,
    "blank"   : 6,
    "auxillary_text"   : 7,
    "auxilary_text"   : 7,
    "auxillary_image": 4, 
    "auxillary_table": 5,
    
}

# Anylabeling-->YOLO format

In [38]:
%cd /content
!mkdir CoCodata
!mkdir CoCodata/images CoCodata/labels
!mkdir CoCodata/labels/train CoCodata/labels/val CoCodata/labels/test
!mkdir CoCodata/images/train CoCodata/images/val CoCodata/images/test

/content


In [39]:
%cd /content

/content


In [40]:
import os
import json

In [62]:
choice_files = [i.path for i in os.scandir("/content/rawdata/raw_train") if i.is_file() and i.path.split(".")[-1]=="json"]

In [64]:
choice_files[0], len(choice_files)

('/content/rawdata/raw_train/scanned353_20.json', 1200)

In [65]:
# json.load(choice_files[0])
choice_dict={}
for i in choice_files:
    f = open(i)
    data = json.load(f)
    choice_dict[data['imagePath'].replace(".jpg", ".txt").replace(".png", ".txt")] = [{'label':category_map[e['label']],
                                                              'bbox':[
                                                                  (e["points"][0][0]+e["points"][1][0])/(2*data["imageWidth"]),
                                                                  (e["points"][0][1]+e["points"][1][1])/(2*data["imageHeight"]),
                                                                  (e["points"][1][0]-e["points"][0][0])/(data["imageWidth"]),
                                                                  (e["points"][1][1]-e["points"][0][1])/(data["imageHeight"])
                                                              ]} for e in data['shapes']]


In [44]:
len(choice_dict.keys())

1200

In [ ]:
choice_dict

In [ ]:
tye = "val"
import os.path as osp
txt_folder = f"/content/CoCodata/labels/{tye}"
# txt_list  = [i.path for i in os.scandir(txt_folder) if i.path.split("/")[-1] in choice_dict.keys()]
txt_list  = [k for k in choice_dict.keys()]
print(len(txt_list))

for x in txt_list:
    data = []
    # with open(x, "r") as f:
    #     for line in f:
    #         line = line.strip()
    #         if line[0] == "3":
    #             print("drop--->", line)
    #             continue

    #         else:
    #             data += [line]
    # new_choices = choice_dict[x.split("/")[-1]]
    new_choices = choice_dict[x]

    for item in new_choices:
        # if item['label'] != 3: continue
        text = f"{item['label']} {item['bbox'][0]} {item['bbox'][1]} {item['bbox'][2]} {item['bbox'][3]}"
        data += [text]
    print(data)
    
    with open(osp.join(txt_folder ,x), "w") as f:
        for line in data:
            f.write(line+"\n")      

In [ ]:
!ls

CoCodata  ds1.zip	     sample_data     small_test
coco.zip  publaynet.zip      searchable_pdf  smalltest.zip
content   questiondetection  searchable.zip


In [ ]:
!rm /content/CoCodata/images/train/*

In [ ]:
choice_dict.keys()

In [50]:
import shutil
import os.path as osp

In [60]:
scan_pth = "/content/rawdata/raw_train"
tye = "train"

In [66]:
name_files = [osp.join(scan_pth, k.replace(".txt", ".png")) for k in choice_dict.keys()]

In [67]:
name_files[4]

'/content/rawdata/raw_train/scanned336_13.png'

In [68]:
desfolder = f"/content/CoCodata/images/{tye}"
for s in name_files:
    shutil.copy(s, osp.join(desfolder, s.split("/")[-1]))

In [ ]:
!cp /content/CoCodata/images/train /content/content/QuestFormCoco/images -r

In [ ]:
!cp /content/CoCodata/labels/train /content/content/QuestFormCoco/labels -r

# Anylabeling-> COCO

In [29]:
!pip install labelme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 8.2 MB/s eta 0:00:00
  Created wheel for labelme: filename=labelme-5.2.1-py3-none-any.whl size=1466726 sha256=69a784db6866c51d6f0bfb60bfe40a1c76fb08f65e8c75ca085e9b3f225ef88d
  Stored in directory: /root/.cache/pip/wheels/de/44/e2/f05944b4d1af582c7d28ab2d1372a57f63b24576f28a4aa0c3
Successfully built labelme


In [30]:
#@title funtion
#!/usr/bin/env python

import argparse
import collections
import datetime
import glob
import json
import os
import os.path as osp
import sys
import uuid

import imgviz
import numpy as np

import labelme

try:
    import pycocotools.mask
except ImportError:
    print("Please install pycocotools:\n\n    pip install pycocotools\n")
    sys.exit(1)


def main(args):
    if osp.exists(args.output_dir):
        print("Output directory already exists:", args.output_dir)
        sys.exit(1)
    os.makedirs(args.output_dir)
    os.makedirs(osp.join(args.output_dir, args.image_folder))
    if not args.noviz:
        os.makedirs(osp.join(args.output_dir, "Visualization"))
    print("Creating dataset:", args.output_dir)

    now = datetime.datetime.now()

    data = dict(
        info=dict(
            description=None,
            url=None,
            version=None,
            year=now.year,
            contributor=None,
            date_created=now.strftime("%Y-%m-%d %H:%M:%S.%f"),
        ),
        licenses=[
            dict(
                url=None,
                id=0,
                name=None,
            )
        ],
        images=[
            # license, url, file_name, height, width, date_captured, id
        ],
        type="instances",
        annotations=[
            # segmentation, area, iscrowd, image_id, bbox, category_id, id
        ],
        categories=[
            # supercategory, id, name
        ],
    )

    class_name_to_id = {}
    for i, line in enumerate(open(args.labels).readlines()):
        class_id = i - 1  # starts with -1
        class_name = line.strip()
        if class_id == -1:
            assert class_name == "__ignore__"
            continue
        class_name_to_id[class_name] = class_id
        data["categories"].append(
            dict(
                supercategory=None,
                id=class_id,
                name=class_name,
            )
        )

    out_ann_file = osp.join(args.output_dir, "annotations.json")
    label_files = glob.glob(osp.join(args.input_dir, "*.json"))
    for image_id, filename in enumerate(label_files):
        print("Generating dataset from:", filename)

        label_file = labelme.LabelFile(filename=filename)

        base = osp.splitext(osp.basename(filename))[0]
        out_img_file = osp.join(args.output_dir, args.image_folder, base + ".jpg")

        img = labelme.utils.img_data_to_arr(label_file.imageData)
        imgviz.io.imsave(out_img_file, img)
        data["images"].append(
            dict(
                license=0,
                url=None,
                file_name=osp.relpath(out_img_file, osp.dirname(out_ann_file)),
                height=img.shape[0],
                width=img.shape[1],
                date_captured=None,
                id=image_id,
            )
        )

        masks = {}  # for area
        segmentations = collections.defaultdict(list)  # for segmentation
        for shape in label_file.shapes:
            points = shape["points"]
            label = shape["label"]
            group_id = shape.get("group_id")
            shape_type = shape.get("shape_type", "polygon")
            mask = labelme.utils.shape_to_mask(
                img.shape[:2], points, shape_type
            )

            if group_id is None:
                group_id = uuid.uuid1()

            instance = (label, group_id)

            if instance in masks:
                masks[instance] = masks[instance] | mask
            else:
                masks[instance] = mask

            if shape_type == "rectangle":
                (x1, y1), (x2, y2) = points
                x1, x2 = sorted([x1, x2])
                y1, y2 = sorted([y1, y2])
                points = [x1, y1, x2, y1, x2, y2, x1, y2]
            if shape_type == "circle":
                (x1, y1), (x2, y2) = points
                r = np.linalg.norm([x2 - x1, y2 - y1])
                # r(1-cos(a/2))<x, a=2*pi/N => N>pi/arccos(1-x/r)
                # x: tolerance of the gap between the arc and the line segment
                n_points_circle = max(int(np.pi / np.arccos(1 - 1 / r)), 12)
                i = np.arange(n_points_circle)
                x = x1 + r * np.sin(2 * np.pi / n_points_circle * i)
                y = y1 + r * np.cos(2 * np.pi / n_points_circle * i)
                points = np.stack((x, y), axis=1).flatten().tolist()
            else:
                points = np.asarray(points).flatten().tolist()

            segmentations[instance].append(points)
        segmentations = dict(segmentations)

        for instance, mask in masks.items():
            cls_name, group_id = instance
            if cls_name not in class_name_to_id:
                continue
            cls_id = class_name_to_id[cls_name]

            mask = np.asfortranarray(mask.astype(np.uint8))
            mask = pycocotools.mask.encode(mask)
            area = float(pycocotools.mask.area(mask))
            bbox = pycocotools.mask.toBbox(mask).flatten().tolist()

            data["annotations"].append(
                dict(
                    id=len(data["annotations"]),
                    image_id=image_id,
                    category_id=cls_id,
                    segmentation=segmentations[instance],
                    area=area,
                    bbox=bbox,
                    iscrowd=0,
                )
            )

        if not args.noviz:
            viz = img
            if masks:
                labels, captions, masks = zip(
                    *[
                        (class_name_to_id[cnm], cnm, msk)
                        for (cnm, gid), msk in masks.items()
                        if cnm in class_name_to_id
                    ]
                )
                viz = imgviz.instances2rgb(
                    image=img,
                    labels=labels,
                    masks=masks,
                    captions=captions,
                    font_size=15,
                    line_width=2,
                )
            out_viz_file = osp.join(
                args.output_dir, "Visualization", base + ".jpg"
            )
            imgviz.io.imsave(out_viz_file, viz)

    with open(out_ann_file, "w") as f:
        json.dump(data, f)




In [34]:
with open("labels.txt", "w") as f:
    text = """__ignore__
heading
question
subquestion
choice
image
table
blank
auxillary_text
"""
    f.write(text)
class Cfg():
    def __init__(self):
        self.input_dir = "/content/rawdata/raw_val"#"/content/searchable_pdf"
        self.output_dir = "/content/mixCOCOformat_val"
        self.labels = "labels.txt"
        self.image_folder = "val"
        self.noviz = True
args = Cfg()

In [ ]:
!rm -rf /content/mixCOCOformat_train

In [ ]:
main(args)

In [9]:
os.remove("/content/mix/scanned310_5.json")

# COCO --> PubLayNet

In [ ]:
%cd /content
!mkdir mixPublaynet
!mkdir mixPublaynet/train mixPublaynet/test mixPublaynet/val


/content


In [ ]:
#@title convert_json
import json

def coco_to_publaynet(coco_file, publaynet_file):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)
    
    publaynet_data = {
        'images': [],
        'annotations': [],
        'categories': coco_data['categories']
    }
    
    image_id_mapping = {}
    annotation_id = 1
    
    for image in coco_data['images']:
        image_id = image['id']
        image_id_mapping[image_id] = len(publaynet_data['images']) + 1
        
        publaynet_image = {
            'id': image_id_mapping[image_id],
            'file_name': image['file_name'],
            'width': image['width'],
            'height': image['height']
        }
        
        publaynet_data['images'].append(publaynet_image)
    
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        if image_id in image_id_mapping:
            publaynet_annotation = {
                'id': annotation_id,
                'image_id': image_id_mapping[image_id],
                'category_id': annotation['category_id'],
                'bbox': annotation['bbox'],
                'segmentation': annotation['segmentation'],
                'area': annotation['area'],
                'iscrowd': annotation['iscrowd']
            }
            
            publaynet_data['annotations'].append(publaynet_annotation)
            annotation_id += 1
    
    with open(publaynet_file, 'w') as f:
        json.dump(publaynet_data, f)




In [ ]:
# Usage
coco_file = '/content/mixCOCOformat/val.json'
publaynet_file = '/content/mixPublaynet/val.json'

coco_to_publaynet(coco_file, publaynet_file)

In [ ]:
!cp /content/mixCOCOformat/train /content/mixPublaynet -rf

In [ ]:
!cp /content/mixCOCOformat/val /content/mixPublaynet -rf

# COCO->YOLO

In [ ]:
!mkdir rawdata

In [17]:
!rm /content/CoCodata/labels/train/*

In [6]:
%cd /content
!mkdir CoCodata
!mkdir CoCodata/images CoCodata/labels
!mkdir CoCodata/labels/train CoCodata/labels/val CoCodata/labels/test
!mkdir CoCodata/images/train CoCodata/images/val CoCodata/images/test

/content


In [19]:
import json

# Load COCO annotation file
coco_annotations = json.load(open('/content/content/mixCOCOformat/val.json'))

# Define class mapping dictionary
class_mapping = category_map = {
    "heading": 0,  # person
    "question": 1,  # bicycle
    "subquestion": 2,
    "choice"  : 3,
    "image"   : 4,
    "table"   : 5,
    "blank"   : 6,
    "auxillary_text"   : 7,   
}
img_dict = {i['id']:i['file_name'].split("/")[-1] for i in coco_annotations["images"]}
hw_dict = {i['id']:(i['height'], i['width']) for i in coco_annotations["images"]}
# Iterate through each annotation
for annotation in coco_annotations['annotations']:
    image_id = annotation['image_id']
    file_name = img_dict[image_id].replace(".jpg", ".txt")
    image_width = hw_dict[image_id][1]
    image_height = hw_dict[image_id][0]
    category_id = annotation['category_id']
    bbox = annotation['bbox']

    # Convert bounding box to YOLO format
    x_center = (bbox[0] + bbox[2] / 2) / image_width
    y_center = (bbox[1] + bbox[3] / 2) / image_height
    box_width = bbox[2] / image_width
    box_height = bbox[3] / image_height

    # # Get class index from class mapping
    # class_index = class_mapping[category_id]

    # Write YOLO annotation to file
    with open(f'/content/CoCodata/labels/val/{file_name}', 'a') as f:
        f.write(f'{category_id} {x_center} {y_center} {box_width} {box_height}\n')


In [21]:
!cp /content/content/mixCOCOformat/val /content/CoCodata/images/ -rf

# ZIP file

In [69]:
!zip mixYOLO.zip /content/CoCodata -r


  adding: content/CoCodata/ (stored 0%)
  adding: content/CoCodata/labels/ (stored 0%)
  adding: content/CoCodata/labels/val/ (stored 0%)
  adding: content/CoCodata/labels/val/searchable795_0.txt (deflated 47%)
  adding: content/CoCodata/labels/val/searchable711_7.txt (deflated 53%)
  adding: content/CoCodata/labels/val/scanned312_18.txt (deflated 49%)
  adding: content/CoCodata/labels/val/scanned341_34.txt (deflated 49%)
  adding: content/CoCodata/labels/val/scanned312_29.txt (deflated 49%)
  adding: content/CoCodata/labels/val/scanned338_9.txt (deflated 51%)
  adding: content/CoCodata/labels/val/scanned367_8.txt (deflated 51%)
  adding: content/CoCodata/labels/val/scanned308_9.txt (deflated 50%)
  adding: content/CoCodata/labels/val/searchable693_3.txt (deflated 52%)
  adding: content/CoCodata/labels/val/scanned310_8.txt (deflated 50%)
  adding: content/CoCodata/labels/val/scanned351_10.txt (deflated 47%)
  adding: content/CoCodata/labels/val/searchable799_3.txt (deflated 51%)
  addi

In [37]:
!ls -lh

total 440M
-rw-r--r-- 1 root root   80 May 28 02:35 labels.txt
drwxr-xr-x 2 root root 124K May 28 02:15 mix
drwxr-xr-x 5 root root 4.0K May 28 02:36 mixCOCOformat
drwxr-xr-x 3 root root 4.0K May 28 02:36 mixCOCOformat_val
-rw-r--r-- 1 root root 140M May 28 02:37 mixCOCO.zip
-rw-r--r-- 1 root root 300M May 28 02:13 mix.zip
drwxr-xr-x 4 root root 4.0K May 28 02:17 rawdata
drwxr-xr-x 1 root root 4.0K May 25 13:42 sample_data
